In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import matplotlib.animation as animation

In [2]:
# finds the closest node from a certain position
def closest_node(position, nodes):
    diff = np.subtract(nodes, position)
    
    errors = np.sum(np.square(diff), axis = 1)
    closest = np.argmin(errors)
    
    return nodes[closest], closest

In [3]:
def voronoi_mapping(grid_resolution, cell_count):
    
    
    nodes = []
    random.seed(1373)
    for i in range(cell_count):
        # creates a node at a random location
        curr_x = random.random()
        curr_y = random.random()
        curr_node = (curr_x, curr_y)
        nodes.append(curr_node)
        
    # iterates over a square grid whose resolution is grid_resolution.
    # At each point in the grid, determines which node is closest
    axis = np.linspace(0, 1, grid_resolution, endpoint = False)
    voronoi_map = []
    for x in axis:
        temp_map = []
        for y in axis:
            curr_position = [x, y]
            voro_node, node_index = closest_node(curr_position, nodes)
            temp_map.append(node_index)
            
        voronoi_map.append(temp_map)
        
    return voronoi_map, nodes
            

In [4]:
# creates and returns a dictionary of a cell's adjacencies

def gen_adj_dict(voronoi_map, nodes):
    grid_resolution = len(voronoi_map)
    cell_count = len(nodes)
    
    # each cell's adjacencies are stored in a set
    adj_dict = {i : set() for i in range(cell_count)}
    
    # iterates over the map in the y direction
    # detects when a change in cells occurs, and 
    # documents that as an adjacency
    prev = voronoi_map[0][0]
    for x in range(grid_resolution): # for every x value
        prev_node = voronoi_map[x][0]
        for y in range(grid_resolution): # iterates in the y direction
            curr_node = voronoi_map[x][y]
            if curr_node != prev_node: # detects changes in the closest node
                adj_dict[curr_node].add(prev_node)
                adj_dict[prev_node].add(curr_node)
            prev_node = curr_node
            
    for y in range(grid_resolution): # for every y value
        prev_node = voronoi_map[0][y]
        for x in range(grid_resolution): # iterates in the x direction
            curr_node = voronoi_map[x][y]
            if curr_node != prev_node: # detects changes in the closest node
                adj_dict[curr_node].add(prev_node)
                adj_dict[prev_node].add(curr_node)
            prev_node = curr_node
            
    return adj_dict

In [5]:
# updates the animation plot
def update_plot(voronoi_map, node_state):
    grid_resolution = len(voronoi_map)
    plot_grid = np.zeros((grid_resolution, grid_resolution))
    
    for x in range(grid_resolution):
        for y in range(grid_resolution):
            curr_node = voronoi_map[x][y]
            if node_state[curr_node]:
                plot_grid[x][y] = ON
                
    return plot_grid

In [6]:
grid_resolution = 400
cell_count = 800

voronoi_map, nodes = voronoi_mapping(400, 800)
adj_dict = gen_adj_dict(voronoi_map, nodes)

In [8]:
delay = 7
beat_rate = 25
random.seed(1373)
seed_node = 200
ON = 255
OFF = 0

# grid_resolution = 400
# cell_count = 800
# delay = 5
# beat_rate = 20
# random.seed(1373)
# seed_node = 200
# ON = 255
# OFF = 0


node_cooldown = {i : 0 for i in range(cell_count)}
node_state = {i : OFF for i in range(cell_count)}

# this init function looks useless, but for some reason,
# the renderer starts on frame 3 without it. With this
# function, it starts on frame 2
def init():
    return mat

def transition(data):
    
    # decrements the cooldown of each node by one
    for node in node_cooldown:
        if node_cooldown[node] > 0:
            node_cooldown[node] -= 1
    
    node_state_copy = node_state.copy()
    for node in node_state.keys():
        
        # if a node is activated
        if node_state_copy[node] == ON:
            # activate its neighbors, if they aren't cooling down
            for neighbor in adj_dict[node]:
                if node_cooldown[neighbor] == 0:
                    node_cooldown[neighbor] = delay
                    node_state[neighbor] = ON
            # deactivate that node
            node_state[node] = OFF

    
     # reignites a random node one every beat_rate frames
    if (data + 1) % beat_rate == 0:
        beat_seed = int(random.random() * cell_count)
        node_state[beat_seed] = ON
        node_cooldown[beat_seed] = delay
        
    plot_grid = update_plot(voronoi_map, node_state)
    
    # updates the plot's data, allowing the renderer to redraw the plot
    mat.set_data(plot_grid)

    return mat

node_state[seed_node] = ON
node_cooldown[seed_node] = delay
plot_grid = update_plot(voronoi_map, node_state)

fig, ax = plt.subplots(figsize = (6,6))
mat = ax.matshow(plot_grid, cmap="Greys_r")
ani = animation.FuncAnimation(fig, transition, interval = 50, save_count=200, init_func = init)
plt.close(fig)

from IPython.display import HTML
HTML(ani.to_jshtml())

### 